*Import Libraries*

In [4]:
from tensorflow import keras      # print(keras.__version__, tf.__version__)
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, preprocessing

*Step 1. Load data*<br/>
[Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html)

In [5]:
data = datasets.fetch_california_housing()
print(f"Data.keys: {data.keys()}")                                      # Information about dataset
print(f"\n Inputs:{data['data']}")                                      # Input data
print(f"\n Labels:{data['target']}")                                    # Targets or Labels
print(f"\n Target_names:{data['feature_names']}")                       # Features name

Data.keys: dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

 Inputs:[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]

 Labels:[4.526 3.585 3.521 ... 0.923 0.847 0.894]

 Target_names:['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [6]:
input_data = data['data']                                         # Input data
labels_data = data['target']                                      # Targets or Labels
print(f"input_data.shape: {input_data.shape}")
print(f"labels.shape: {labels_data.shape}")

input_data.shape: (20640, 8)
labels.shape: (20640,)


*Step 2: [Split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)*<br/>
Split dataset into random train and test subsets<br/>
Split train into random train and validation subsets<br/>

In [7]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(input_data, labels_data, test_size=0.25)  # Split data into train & test
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(x_train, y_train, test_size=0.25) # Split train into train & validation
print(f"x_train.shape:{x_train.shape}")
print(f"x_test.shape:{x_test.shape}")
print(f"x_validation.shape:{x_validation.shape}")
print(f"y_train.shape:{y_train.shape}")
print(f"y_test.shape:{y_test.shape}")
print(f"y_validation.shape:{y_validation.shape}")

x_train.shape:(11610, 8)
x_test.shape:(5160, 8)
x_validation.shape:(3870, 8)
y_train.shape:(11610,)
y_test.shape:(5160,)
y_validation.shape:(3870,)


*Step 3: [Normalize data](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)*


In [8]:
norm = preprocessing.StandardScaler()
x_train_norm = norm.fit_transform(x_train)
x_validation_norm = norm.transform(x_validation)
x_test_norm = norm.transform(x_test)

*[Wrappers for the Scikit-Learn API](https://faroit.com/keras-docs/1.0.6/scikit-learn-api/)*<br/>
*[GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)*<br/>
*[Hyperas](https://github.com/maxpumperla/hyperas)*<br/>
*[KerasTuner](https://keras.io/keras_tuner/)*<br/>
*[Hyperband Tuner](https://keras.io/api/keras_tuner/tuners/hyperband/) ---> [Github](https://github.com/zygmuntz/hyperband)*<br/>
*[Article](https://arxiv.org/pdf/1803.09820.pdf)*

In [18]:
# ------------------------------------------- Build the model & compile ------------------------------------------------
def deep_model(x_train_norm, num_hidden_layers=1, num_neurons=50, lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=x_train_norm.shape[1]))
    for i in range(num_hidden_layers):
        model.add(keras.layers.Dense(num_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

keras_scikit_reg = keras.wrappers.sklearn.KerasRegressor(build_fn=deep_model)
param_grid = {"number_of_hidden_layers": [1, 3, 5], "number_of_neurons": [50, 70, 100], "lr": [0.001, 0.01, 0.1]}
keras_sk_reg_gs = model_selection.GridSearchCV(keras_scikit_reg, param_grid)
# ----------------------------------------- Fit, evaluate & predictModel ----------------------------------------------
keras_sk_reg_gs.fit(x_train_norm, y_train, epochs=2, validation_data=(x_validation_norm, y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5)])
keras_sk_reg_gs.param_grid
keras_sk_reg_gs.best_params_
final_model = keras_sk_reg_gs.best_estimator_.model

AttributeError: module 'keras.api._v2.keras' has no attribute 'wrappers'